In [3]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Directory containing data files
data_dir = r'C:\Users\User\Documents\Lie detect data\AugmentedEEGData'

# Function to load and label data
def load_data(data_dir):
    X = []
    y = []
    
    for file_name in os.listdir(data_dir):
        if file_name.endswith('.pkl'):
            file_path = os.path.join(data_dir, file_name)
            data = pd.read_pickle(file_path)
            label = 0 if 'lie' in file_name else 1
            X.append(data)
            y.extend([label] * data.shape[0])
    
    X = np.vstack(X)
    y = np.array(y)
    return X, y

# Load and label data
X, y = load_data(data_dir)

# Normalize data
scaler = StandardScaler()
X = X.reshape(X.shape[0], -1)  # Reshape to (n_samples, n_features)
X = scaler.fit_transform(X)
X = X.reshape(-1, 65, 125)  # Reshape back to (n_samples, n_channels, n_times)

# Shuffle data
indices = np.random.permutation(X.shape[0])
X, y = X[indices], y[indices]

# Define dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset and data loaders
dataset = EEGDataset(X, y)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define EEGNet model
class EEGNet(nn.Module):
    def __init__(self, num_classes=2):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (1, 63), padding='same')  # Changed (1, 64) to (1, 63)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.depthwiseConv2d = nn.Conv2d(16, 32, (65, 1), groups=16, padding='same')
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.activation = nn.ELU()
        self.pooling = nn.AvgPool2d((1, 4))
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 65 * 31, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.depthwiseConv2d(x)
        x = self.batchnorm2(x)
        x = self.activation(x)
        x = self.pooling(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EEGNet(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# Training loop with early stopping
num_epochs = 100
best_val_loss = float('inf')
patience = 10
trigger_times = 0

# Directory to save the model
model_save_dir = r'C:\Users\User\Documents\Lie detect data\Model'
os.makedirs(model_save_dir, exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
    
    scheduler.step(val_loss)
    
    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        best_model_path = os.path.join(model_save_dir, 'BEST_OSW_EEGNet.pth')
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'val_loss': best_val_loss,
        }, best_model_path)
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break

# Save the final model
final_model_path = os.path.join(model_save_dir, 'FINAL_OSW_EEGNet.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
    'val_loss': val_loss,
}, final_model_path)

# Test the model
model.eval()
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
        all_labels.extend(y_batch.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_loss /= len(test_loader)
test_accuracy = correct / total

# Calculate additional metrics
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)

print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}, AUC: {auc}')
print('Confusion Matrix:')
print(conf_matrix)


Epoch 1/100, Loss: 1.249487687434469, Validation Loss: 1.2309532523155213, Validation Accuracy: 0.7857142857142857
Epoch 2/100, Loss: 0.966152296889396, Validation Loss: 1.8516242623329162, Validation Accuracy: 0.85
Epoch 3/100, Loss: 0.763202484342314, Validation Loss: 0.7839669406414032, Validation Accuracy: 0.8285714285714286
Epoch 4/100, Loss: 0.4376492975723176, Validation Loss: 0.7140443116426468, Validation Accuracy: 0.8642857142857143
Epoch 5/100, Loss: 0.2839018309799333, Validation Loss: 0.6308681726455688, Validation Accuracy: 0.8714285714285714
Epoch 6/100, Loss: 0.1998067611296262, Validation Loss: 1.1516806751489639, Validation Accuracy: 0.8642857142857143
Epoch 7/100, Loss: 0.3147124865402778, Validation Loss: 1.15094259865582, Validation Accuracy: 0.8714285714285714
Epoch 8/100, Loss: 0.39899506100586485, Validation Loss: 1.4500081419944764, Validation Accuracy: 0.8214285714285714
Epoch 9/100, Loss: 0.4360686703806832, Validation Loss: 0.7971711218357086, Validation Acc